In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Generate a random matrix A
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A

In [3]:
sample_size = 100000

diag_estimate = np.zeros(n)
tk = np.zeros(n)
qk = np.zeros(n)

for j in range(sample_size):
    
    # Draw random vector
    vk = np.random.choice([-1, 1], size=n)

    # Update tk
    tk = tk + ((A @ vk) * vk)

    # Update qk
    qk = qk + (vk*vk)

    # Update diag_estimate
    diag_estimate = tk / qk

In [4]:
(np.linalg.norm( np.diag(A) - diag_estimate , ord=2)**2)/n

9.864831662658363

In [ ]:
def naive_diag(A, sample_size=1000):
    """Naive unbiased estimator for the diagonal of a matrix, see [5].
    """

    # Get shape
    n = A.shape[0]

    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = np.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

# Explicit probe?

In [6]:
def explicit_diag_probe(A):
    """Computes the diagonal of A using an explicit probe. Requires exactly n matvecs and rmatvecs with A.
    """

    # Setup
    n = A.shape[0]
    diagonal = np.zeros(n)

    for j in range(n):

        # jth column of the identity
        w = np.zeros(n)
        w[j] = 1.0

        # Compute w^T A w
        diagonal[j] = w.T @ A @ w

    return diagonal

In [7]:
explicit_diag_probe(A)

array([1025.11478784, 1056.50980002, 1076.9164419 , 1009.48791874,
       1048.16560964,  959.93411077, 1035.6242839 , 1029.58367682,
        984.10238742,  993.11142698,  990.69554211,  965.04694637,
       1027.47981527, 1047.39880032,  970.08334629,  977.91386144,
       1045.30756413,  975.61862357,  974.59327804, 1071.65089356,
       1030.65241408, 1094.7979908 ,  999.43333907,  978.66523031,
       1064.88413702, 1069.71482696,  966.13685778, 1020.66408521,
        965.3875889 , 1044.92004135, 1030.31917365, 1008.25928482,
       1038.98136824, 1032.91829121,  997.98050451, 1010.64270855,
       1017.28844226,  966.42089143,  967.18082444,  922.42580821,
       1050.13493673, 1023.71847583,  932.97995415, 1064.6596305 ,
       1006.51760743,  942.1758074 , 1037.48263977,  954.22345525,
       1022.49216184, 1005.65500563,  920.82423911,  953.18228166,
       1043.89648805,  973.94325974, 1030.88025438, 1123.78994281,
       1066.01654487, 1064.72311789, 1037.05726989,  933.15857

# Hadamard matrices method?